# Семинар 4: Deep Deterministic Policy Gradient

На этом семинаре мы будем обучать нейронную сеть на фреймворке __pytorch__ с помощью алгоритма Deep Deterministic Policy Gradient.

## Теория

Deep Deterministic Policy Gradient (DDPG) - это алгоритм, который одновременно учит Q-функцию и стратегию. Он использует off-policy данные и уравнения Беллмана для обучения Q-функции, а Q-функция используется для обучения стратегии.

Данный подход тесно связан с Q-обучением и мотивирован следующей идеей: если вы знаете оптимальную функцию action-value $Q^*(s,a)$, тогда для конкретного состояния, оптимальное действие $a^*(s)$ может быть найдено решением: 

$$a^*(s) = \arg \max_a Q^*(s,a).$$

Для сред с дискретным пространством действий - это легко, вычисляем полезности для каждого из действий, а потом берем максимум. Для непрерывных действий - это сложная оптимизационная задача.

DDPG чередует обучение аппроксиматора $Q^*(s,a)$ с обучением аппроксиматора  $a^*(s)$, и делает это специальным образом именно для непрерывных (continuous) сред, что отражается в том как алгоритм вычисляет $\max_a Q^*(s,a)$.
Поскольку пространство действий непрерывно, предполагается, что функция $Q^*(s,a)$ дифференцируема по аргументу действия. Это позволяет нам установить эффективное правило обучения на основе градиента для стратегии $\mu(s)$.

<img src="https://spinningup.openai.com/en/latest/_images/math/5811066e89799e65be299ec407846103fcf1f746.svg">

In [ ]:
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

## Environment
### Нормализация пространства действий

In [ ]:
class NormalizedActions(gym.ActionWrapper):

    def action(self, action):
        low_bound   = self.action_space.low
        upper_bound = self.action_space.high
        
        action = low_bound + (action + 1.0) * 0.5 * (upper_bound - low_bound)
        action = np.clip(action, low_bound, upper_bound)
        
        return action

    def reverse_action(self, action):
        pass

### Исследование - GaussNoise
#### Добавляем Гауссовский шум к действиям детерминированной стратегии

In [ ]:
class GaussNoise:

    def __init__(self, sigma):
        super().__init__()

        self.sigma = sigma

    def get_action(self, action):
        noisy_action = np.random.normal(action, self.sigma)
        return noisy_action

<h2>Оригинальная статья: Continuous control with deep reinforcement learning <a href="https://arxiv.org/abs/1509.02971">Arxiv</a></h2>

In [ ]:
class ValueNetwork(nn.Module):
    def __init__(
        self, 
        num_inputs, 
        num_actions, 
        hidden_size, 
        init_w=3e-3
    ):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(num_inputs + num_actions, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
        )
        self.head = nn.Linear(hidden_size, 1)
        
        self.head.weight.data.uniform_(-init_w, init_w)
        self.head.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state, action):
        x = torch.cat([state, action], 1)
        x = self.net(x)
        x = self.head(x)
        return x


class PolicyNetwork(nn.Module):
    def __init__(
        self, 
        num_inputs, 
        num_actions, 
        hidden_size, 
        init_w=3e-3
    ):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
        )
        self.head = nn.Linear(hidden_size, num_actions)
        
        self.head.weight.data.uniform_(-init_w, init_w)
        self.head.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state):
        x = state
        x = self.net(x)
        x = self.head(x)
        return x
    
    def get_action(self, state):
        state  = torch.tensor(state, dtype=torch.float32)\
            .unsqueeze(0).to(device)
        action = self.forward(state)
        action = action.detach().cpu().numpy()[0]
        return action

<h2>DDPG обновление</h2>

In [ ]:
def ddpg_update(
    state, 
    action, 
    reward, 
    next_state, 
    done, 
    gamma = 0.99,
    min_value=-np.inf,
    max_value=np.inf,
    soft_tau=1e-2,
):  
    state      = torch.tensor(state, dtype=torch.float32).to(device)
    next_state = torch.tensor(next_state, dtype=torch.float32).to(device)
    action     = torch.tensor(action, dtype=torch.float32).to(device)
    reward     = torch.tensor(reward, dtype=torch.float32).unsqueeze(1).to(device)
    done       = torch.tensor(np.float32(done)).unsqueeze(1).to(device)

    policy_loss = value_net(state, policy_net(state))
    policy_loss = -policy_loss.mean()

    next_action    = target_policy_net(next_state)
    target_value   = target_value_net(next_state, next_action.detach())
    expected_value = reward + (1.0 - done) * gamma * target_value
    expected_value = torch.clamp(expected_value, min_value, max_value)

    value = value_net(state, action)
    value_loss = value_criterion(value, expected_value.detach())


    policy_optimizer.zero_grad()
    policy_loss.backward()
    policy_optimizer.step()

    value_optimizer.zero_grad()
    value_loss.backward()
    value_optimizer.step()

    for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
            target_param.data.copy_(
                target_param.data * (1.0 - soft_tau) + param.data * soft_tau
            )

    for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
            target_param.data.copy_(
                target_param.data * (1.0 - soft_tau) + param.data * soft_tau
            )

## Память повторов (replay buffer)

In [ ]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done
    
    def __len__(self):
        return len(self.buffer)

---

In [ ]:
batch_size  = 128

def generate_session(t_max=1000, train=False):
    """эпизод взаимодействие агента со средой, а также вызов процесса обучения"""
    total_reward = 0
    state = env.reset()

    for t in range(t_max):
        action = policy_net.get_action(state)
        if train:
            action = noise.get_action(action)
        next_state, reward, done, _ = env.step(action)

        if train:
            replay_buffer.push(state, action, reward, next_state, done)
            if len(replay_buffer) > batch_size:
                states, actions, rewards, next_states, dones = \
                    replay_buffer.sample(batch_size)
                ddpg_update(states, actions, rewards, next_states, dones)

        total_reward += reward
        state = next_state
        if done:
            break

    return total_reward

In [ ]:
env = NormalizedActions(gym.make("Pendulum-v0"))
noise = GaussNoise(sigma=0.3)

state_dim  = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
hidden_dim = 256

value_net  = ValueNetwork(state_dim, action_dim, hidden_dim).to(device)
policy_net = PolicyNetwork(state_dim, action_dim, hidden_dim).to(device)

target_value_net  = ValueNetwork(state_dim, action_dim, hidden_dim).to(device)
target_policy_net = PolicyNetwork(state_dim, action_dim, hidden_dim).to(device)

for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
    target_param.data.copy_(param.data)

for target_param, param in zip(target_policy_net.parameters(), policy_net.parameters()):
    target_param.data.copy_(param.data)
    
    
value_lr  = 1e-3
policy_lr = 1e-4

value_optimizer  = optim.Adam(value_net.parameters(),  lr=value_lr)
policy_optimizer = optim.Adam(policy_net.parameters(), lr=policy_lr)

value_criterion = nn.MSELoss()

replay_buffer_size = 10000
replay_buffer = ReplayBuffer(replay_buffer_size)

In [ ]:
max_steps   = 500

valid_mean_rewards = []
for i in range(100):    
    session_rewards_train = [
        generate_session(t_max=max_steps, train=True) 
        for _ in range(10)
    ]
    session_rewards_valid = [
        generate_session(t_max=max_steps, train=False) 
        for _ in range(10)
    ]
    print(
        "epoch #{:02d}\tmean reward (train) = {:.3f}\tmean reward (valid) = {:.3f}".format(
        i, np.mean(session_rewards_train), np.mean(session_rewards_valid))
    )

    valid_mean_rewards.append(np.mean(session_rewards_valid))
    if len(valid_mean_rewards) > 5 and np.mean(valid_mean_rewards[-5:]) > -200:
        print("Pendulum принят!")
        break

epoch #00	mean reward (train) = -1326.031	mean reward (valid) = -1595.047
epoch #01	mean reward (train) = -1196.696	mean reward (valid) = -1268.831
epoch #02	mean reward (train) = -264.864	mean reward (valid) = -145.502
epoch #03	mean reward (train) = -155.861	mean reward (valid) = -132.042
epoch #04	mean reward (train) = -167.142	mean reward (valid) = -134.034
epoch #05	mean reward (train) = -132.595	mean reward (valid) = -173.017
epoch #06	mean reward (train) = -203.250	mean reward (valid) = -152.431
Pendulum принят!


---

In [ ]:
# record sessions
#import gym.wrappers
#env = gym.wrappers.Monitor(
#    NormalizedActions(gym.make("Pendulum-v0")),
#    directory="videos_ddpg", 
#    force=True)
#sessions = [generate_session(t_max=max_steps, train=False) for _ in range(10)]
#env.close()

---

### Задание 1: обучите алгоритм DDPG на одной из сред с непрерывным пространством действий, которые мы рассматривали на одном из прошлых семинаров. 

### Задание 2: Измените гиперпараметры алгоритма так, чтобы получить лучшую кривую сходимости.   